# Big Data Challenge
## Continuation (Part 2)
### See video game script for part 1

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ 
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http:

In [ ]:
#Connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
#link to use 
#https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz

### Extract

In [2]:
#using pyspark to extract and transform big data
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("apparel_review").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
#load in 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
apparel_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy/MM/dd HH:mm:ss")
apparel_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

### Transform

In [ ]:
#transform dataset to fit the tables in the schema file 
review_df = apparel_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_df.show(5)

In [ ]:
product_df = apparel_df.select(["product_id", "product_title"])
product_df.show(5)

In [ ]:
count = apparel_df.groupBy("customer_id").count()
count_df = count.withColumnRenamed("count", "customer_count")
count_df.show(5)

In [ ]:
vine_df = apparel_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(5)

## Load

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url= endpoint
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [ ]:
 # Write DataFrame to table in RDS

review_df.write.jdbc(url=jdbc_url, table='review_table2', mode=mode, properties=config)

In [ ]:
 # Write DataFrame to table in RDS

product_df.write.jdbc(url=jdbc_url, table='vg_products2', mode=mode, properties=config)

In [ ]:
 # Write DataFrame to table in RDS

count_df.write.jdbc(url=jdbc_url, table='customer_table2', mode=mode, properties=config)

In [ ]:
 # Write DataFrame to table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table2', mode=mode, properties=config)